### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/data/20220507_P183_octopus_SYTOXG/Atlas/P183_536BP_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/data/20220507_P183_octopus_SYTOXG/Atlas/mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "P183_536BP_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

# aff = np.array([[ 1.0, 0, 0],
#                 [ 0, 1.0, 0],
#                 [ 0, 0, 1.0]], dtype='float')
aff = np.array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.76604444, -0.64278761],
       [ 0.        ,  0.64278761,  0.76604444]], dtype="float")
transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/220617_P200_skeletonization/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,4",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=12)
R.set_fixed_image(os.path.join(tmpdir, "P183_536BP_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/220617_P200_skeletonization/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/220617_P200_skeletonization/ants/F2M_,/flash/ReiterU/tmp/220617_P200_skeletonization/ants/Warped.nii.gz,/flash/ReiterU/tmp/220617_P200_skeletonization/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/220617_P200_skeletonization/ants/P183_536BP_10um_masked.nii,/flash/ReiterU/tmp/220617_P200_skeletonization/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/220617_P200_skeletonization/ants/P183_536BP_10um_masked.nii,/flash/ReiterU/tmp/

DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 2DIAGNOSTIC,     1, -2.562413625200e-02, 1.797693134862e+308, 3.1437e+00, 3.1437e+00,
 2DIAGNOSTIC,     2, -2.563668925502e-02, 1.797693134862e+308, 3.1937e+00, 4.9958e-02,
 2DIAGNOSTIC,     3, -2.566583342461e-02, 1.797693134862e+308, 3.2359e+00, 4.2257e-02,
 2DIAGNOSTIC,     4, -2.570234417347e-02, 1.797693134862e+308, 3.2824e+00, 4.6515e-02,
 2DIAGNOSTIC,     5, -2.578729563789e-02, 1.797693134862e+308, 3.3262e+00, 4.3800e-02,
 2DIAGNOSTIC,     6, -2.586268693018e-02, 1.797693134862e+308, 3.3762e+00, 5.0000e-02,
 2DIAGNOSTIC,     7, -2.593768029981e-02, 1.797693134862e+308, 3.4218e+00, 4.5554e-02,
 2DIAGNOSTIC,     8, -2.598134847005e-02, 1.797693134862e+308, 3.4725e+00, 5.0692e-02,
 2DIAGNOSTIC,     9, -2.602979049472e-02, 1.797693134862e+308, 3.5286e+00, 5.6094e-02,
 2DIAGNOSTIC,    10, -2.607762821546e-02, 1.797693134862e+308, 3.5786e+00, 5.0055e-02,
 2DIAGNOSTIC,    11, -2.617494644102e-02, 1.7976

 2DIAGNOSTIC,    41, -3.607808651124e-01, 2.061910482402e-03, 2.4249e+01, 4.4053e-01,
 2DIAGNOSTIC,    42, -3.625319483138e-01, 2.059767897409e-03, 2.4836e+01, 5.8730e-01,
 2DIAGNOSTIC,    43, -3.632559265070e-01, 1.997003805413e-03, 2.5212e+01, 3.7513e-01,
 2DIAGNOSTIC,    44, -3.648659663026e-01, 1.898260298852e-03, 2.5567e+01, 3.5565e-01,
 2DIAGNOSTIC,    45, -3.654038584971e-01, 1.774022466579e-03, 2.5976e+01, 4.0879e-01,
 2DIAGNOSTIC,    46, -3.656492180827e-01, 1.605078261002e-03, 2.6507e+01, 5.3140e-01,
 2DIAGNOSTIC,    47, -3.658982687090e-01, 1.412859346283e-03, 2.6827e+01, 3.1943e-01,
 2DIAGNOSTIC,    48, -3.671090536344e-01, 1.272873549781e-03, 2.7166e+01, 3.3878e-01,
 2DIAGNOSTIC,    49, -3.688522361741e-01, 1.126045095195e-03, 2.7495e+01, 3.2921e-01,
 2DIAGNOSTIC,    50, -3.717095165505e-01, 1.002679901134e-03, 2.7845e+01, 3.5060e-01,
 2DIAGNOSTIC,    51, -3.720786491578e-01, 8.814761677167e-04, 2.8551e+01, 7.0539e-01,
 2DIAGNOSTIC,    52, -3.720859275341e-01, 7.4988904170

 1DIAGNOSTIC,    12, -7.361960219287e-01, 1.924069455697e-03, 1.7248e+01, 1.1312e+00,
 1DIAGNOSTIC,    13, -7.382118372703e-01, 1.643581043547e-03, 1.8316e+01, 1.0679e+00,
 1DIAGNOSTIC,    14, -7.401168427797e-01, 1.426136359858e-03, 1.9416e+01, 1.1006e+00,
 1DIAGNOSTIC,    15, -7.419633619194e-01, 1.244818768015e-03, 2.0628e+01, 1.2121e+00,
 1DIAGNOSTIC,    16, -7.438034396615e-01, 1.100865242817e-03, 2.1816e+01, 1.1876e+00,
 1DIAGNOSTIC,    17, -7.456491256455e-01, 9.800527309046e-04, 2.2931e+01, 1.1150e+00,
 1DIAGNOSTIC,    18, -7.475428804691e-01, 8.812608898031e-04, 2.4054e+01, 1.1231e+00,
 1DIAGNOSTIC,    19, -7.494728959487e-01, 8.044999776205e-04, 2.5077e+01, 1.0225e+00,
 1DIAGNOSTIC,    20, -7.512652188068e-01, 7.459044122435e-04, 2.6095e+01, 1.0188e+00,
 1DIAGNOSTIC,    21, -7.528851868199e-01, 6.967045798099e-04, 2.7186e+01, 1.0905e+00,
 1DIAGNOSTIC,    22, -7.543760068720e-01, 6.506532003358e-04, 2.8333e+01, 1.1472e+00,
 1DIAGNOSTIC,    23, -7.557565020631e-01, 6.0451645357

 1DIAGNOSTIC,   108, -8.082867478284e-01, 2.290935778944e-05, 1.2673e+02, 1.1058e+00,
 1DIAGNOSTIC,   109, -8.086996650538e-01, 2.566221225462e-05, 1.2804e+02, 1.3100e+00,
 1DIAGNOSTIC,   110, -8.090916694917e-01, 2.750007554769e-05, 1.2913e+02, 1.0949e+00,
 1DIAGNOSTIC,   111, -8.095306101736e-01, 2.852834019700e-05, 1.3023e+02, 1.0956e+00,
 1DIAGNOSTIC,   112, -8.099504649898e-01, 2.874520612714e-05, 1.3142e+02, 1.1894e+00,
 1DIAGNOSTIC,   113, -8.104150938835e-01, 2.857983716634e-05, 1.3249e+02, 1.0700e+00,
 1DIAGNOSTIC,   114, -8.108615766368e-01, 2.819351382618e-05, 1.3347e+02, 9.7793e-01,
 1DIAGNOSTIC,   115, -8.112883918326e-01, 2.783479850516e-05, 1.3449e+02, 1.0210e+00,
 1DIAGNOSTIC,   116, -8.117282309710e-01, 2.769166205371e-05, 1.3575e+02, 1.2587e+00,
 1DIAGNOSTIC,   117, -8.121871410313e-01, 2.767961011440e-05, 1.3695e+02, 1.2063e+00,
 1DIAGNOSTIC,   118, -8.126659483344e-01, 2.782212313969e-05, 1.3817e+02, 1.2159e+00,
 1DIAGNOSTIC,   119, -8.131091179027e-01, 2.7880486737

 1DIAGNOSTIC,   204, -8.384748828805e-01, 8.315041184471e-06, 2.3627e+02, 1.2795e+00,
 1DIAGNOSTIC,   205, -8.387276885738e-01, 8.522549758135e-06, 2.3735e+02, 1.0838e+00,
 1DIAGNOSTIC,   206, -8.388650839181e-01, 8.572007614335e-06, 2.3836e+02, 1.0084e+00,
 1DIAGNOSTIC,   207, -8.391247073684e-01, 8.450917942116e-06, 2.3953e+02, 1.1680e+00,
 1DIAGNOSTIC,   208, -8.393884926937e-01, 8.404555683071e-06, 2.4074e+02, 1.2138e+00,
 1DIAGNOSTIC,   209, -8.395224592897e-01, 7.997980372724e-06, 2.4178e+02, 1.0356e+00,
 1DIAGNOSTIC,   210, -8.398000702870e-01, 7.809554474892e-06, 2.4285e+02, 1.0742e+00,
 1DIAGNOSTIC,   211, -8.400390566318e-01, 7.722959138517e-06, 2.4392e+02, 1.0716e+00,
 1DIAGNOSTIC,   212, -8.402425555058e-01, 7.639640044962e-06, 2.4509e+02, 1.1651e+00,
 1DIAGNOSTIC,   213, -8.403568351343e-01, 7.429351315878e-06, 2.4625e+02, 1.1644e+00,
 1DIAGNOSTIC,   214, -8.404524036434e-01, 7.053792026496e-06, 2.4749e+02, 1.2415e+00,
 1DIAGNOSTIC,   215, -8.405190715360e-01, 6.5362762652

 1DIAGNOSTIC,    77, -8.762500838411e-01, 1.840234526585e-06, 8.7928e+02, 7.6765e+00,
 1DIAGNOSTIC,    78, -8.762921994908e-01, 1.558731450278e-06, 8.8727e+02, 7.9860e+00,
 1DIAGNOSTIC,    79, -8.763347678913e-01, 1.492366849813e-06, 8.9509e+02, 7.8157e+00,
 1DIAGNOSTIC,    80, -8.763028433442e-01, 1.315175348789e-06, 9.0319e+02, 8.1089e+00,
 1DIAGNOSTIC,    81, -8.763468402353e-01, 1.352694682160e-06, 9.1170e+02, 8.5095e+00,
 1DIAGNOSTIC,    82, -8.763887918807e-01, 1.571630375657e-06, 9.2043e+02, 8.7260e+00,
 1DIAGNOSTIC,    83, -8.763565070396e-01, 1.664604939933e-06, 9.2882e+02, 8.3925e+00,
 1DIAGNOSTIC,    84, -8.763981473472e-01, 1.634349173820e-06, 9.3749e+02, 8.6697e+00,
 1DIAGNOSTIC,    85, -8.764411141455e-01, 1.753037981072e-06, 9.4584e+02, 8.3481e+00,
 1DIAGNOSTIC,    86, -8.764095443456e-01, 1.759448125357e-06, 9.5384e+02, 7.9992e+00,
 1DIAGNOSTIC,    87, -8.764500252696e-01, 1.637305376077e-06, 9.6197e+02, 8.1258e+00,
 1DIAGNOSTIC,    88, -8.764978553847e-01, 1.7228124558

 1DIAGNOSTIC,    48, -8.833024294189e-01, 1.307116861768e-05, 4.0760e+03, 5.9842e+01,
 1DIAGNOSTIC,    49, -8.833981290409e-01, 1.226303345645e-05, 4.1330e+03, 5.6980e+01,
 1DIAGNOSTIC,    50, -8.834477168696e-01, 1.141641150147e-05, 4.1905e+03, 5.7514e+01,
 1DIAGNOSTIC,    51, -8.835379306259e-01, 1.084489141138e-05, 4.2484e+03, 5.7904e+01,
 1DIAGNOSTIC,    52, -8.835797653616e-01, 1.025321845227e-05, 4.3089e+03, 6.0482e+01,
 1DIAGNOSTIC,    53, -8.836642596626e-01, 9.646947324939e-06, 4.3669e+03, 5.7998e+01,
 1DIAGNOSTIC,    54, -8.837026931401e-01, 9.033540558884e-06, 4.4258e+03, 5.8941e+01,
 1DIAGNOSTIC,    55, -8.837813023290e-01, 8.703661424424e-06, 4.4826e+03, 5.6787e+01,
 1DIAGNOSTIC,    56, -8.838113735319e-01, 8.092876460277e-06, 4.5410e+03, 5.8339e+01,
 1DIAGNOSTIC,    57, -8.838834510237e-01, 7.741526024920e-06, 4.5987e+03, 5.7705e+01,
 1DIAGNOSTIC,    58, -8.839061722318e-01, 7.128415384795e-06, 4.6575e+03, 5.8837e+01,
 1DIAGNOSTIC,    59, -8.839707583313e-01, 6.7800394299

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/220617_P200_skeletonization/ants', 'deigo:/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/analysis'], returncode=0)